# Aula 1 - Buscando a similaridade

In [1]:
url = 'https://raw.githubusercontent.com/Mirlaa/NLP-trabalhando-similaridade-sentencas/refs/heads/main/reviews_zoop.csv'

In [2]:
import pandas as pd

dados = pd.read_csv(url)

dados

,nota_review,review
0,5,Recebi bem antes do prazo estipulado.
1,5,Parabéns lojas Zoop adorei comprar pela Intern...
2,4,aparelho eficiente. no site a marca do aparelh...
3,4,"Mas um pouco ,travando...pelo valor ta Boa.\r\n"
4,5,"Vendedor confiável, produto ok e entrega antes..."
...,...,...
39457,5,Entregou dentro do prazo. O produto chegou em ...
39458,3,"O produto não foi enviado com NF, não existe v..."
39459,5,"Excelente mochila, entrega super rápida. Super..."
39460,1,Solicitei a compra de uma capa de retrovisor c...


In [3]:
dados['review'][1]

'Parabéns lojas Zoop adorei comprar pela Internet seguro e prático Parabéns a todos feliz Páscoa'

In [4]:
dados['review'][8]

'A compra foi realizada facilmente.\r\nA entrega foi efetuada muito antes do prazo dado.\r\nO produto já começou a ser usado e até o presente,\r\nsem problemas.'

In [5]:
dados['review'][11]

'Sempre compro pela Internet e a entrega ocorre antes do prazo combinado, que acredito ser o prazo máximo. No  o prazo máximo já se esgotou e ainda não recebi o produto.'

In [6]:
dados['review'][18]

'otimo vendedor chegou ate antes do prazo , adorei o produto'

## Similaridade entre as palavras

In [7]:
from nltk.tokenize import word_tokenize
from functools import partial

word_tokenize_pt = partial(word_tokenize, language='portuguese')

In [14]:
import nltk

nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [11]:
!pip install gensim

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 69.9 MB/s eta 0:00:00


In [15]:
from gensim.models import Word2Vec

modelo = Word2Vec(sentences=dados['review'].apply(word_tokenize_pt),
                  vector_size=100, min_count=1, window=5,
                  workers=1, seed=45)

In [16]:
modelo.wv.most_similar('entrega')

[('Entrega', 0.7579216361045837),
 ('entregue', 0.7184001207351685),
 ('compra', 0.7105124592781067),
 ('chegou', 0.6817407608032227),
 ('entregou', 0.6612473130226135),
 ('preparar', 0.6325778365135193),
 ('envio', 0.6313653588294983),
 ('sorriso', 0.6210829615592957),
 ('Chegou', 0.6198225021362305),
 ('mercadoria', 0.6146730780601501)]

In [17]:
modelo.wv.most_similar('vendedor')

[('atendimento', 0.8572612404823303),
 ('serviço', 0.8074656128883362),
 ('fornecedor', 0.7500720024108887),
 ('veiculo', 0.7239760756492615),
 ('produto', 0.7152891755104065),
 ('processo', 0.7085978388786316),
 ('parceiro', 0.7057496905326843),
 ('cumpriu', 0.6909526586532593),
 ('trabalho', 0.6848700642585754),
 ('cumpriram', 0.6713030934333801)]

In [18]:
modelo.wv.most_similar('produto')

[('pedido', 0.749972939491272),
 ('relógio', 0.7460570335388184),
 ('vendedor', 0.7152891755104065),
 ('endereço', 0.7050330638885498),
 ('Produto', 0.6978251934051514),
 ('mesmo', 0.6896668076515198),
 ('tempo', 0.6797294616699219),
 ('Mas', 0.6640905737876892),
 ('mercadoria', 0.6585670113563538),
 ('condizia', 0.6553379893302917)]

## Similaridade entre sentenças

In [19]:
dados['review_token'] = dados['review'].apply(word_tokenize_pt)

In [22]:
from gensim.models.doc2vec import TaggedDocument

dados_tag = [TaggedDocument(words=linha['review_token'], tags=[str(i)]) for i, linha in dados.iterrows()]

In [23]:
dados_tag

[TaggedDocument(words=['Recebi', 'bem', 'antes', 'do', 'prazo', 'estipulado', '.'], tags=['0']),
 TaggedDocument(words=['Parabéns', 'lojas', 'Zoop', 'adorei', 'comprar', 'pela', 'Internet', 'seguro', 'e', 'prático', 'Parabéns', 'a', 'todos', 'feliz', 'Páscoa'], tags=['1']),
 TaggedDocument(words=['aparelho', 'eficiente', '.', 'no', 'site', 'a', 'marca', 'do', 'aparelho', 'esta', 'impresso', 'como', '3desinfector', 'e', 'ao', 'chegar', 'esta', 'com', 'outro', 'nome', '...', 'atualizar', 'com', 'a', 'marca', 'correta', 'uma', 'vez', 'que', 'é', 'o', 'mesmo', 'aparelho'], tags=['2']),
 TaggedDocument(words=['Mas', 'um', 'pouco', ',', 'travando', '...', 'pelo', 'valor', 'ta', 'Boa', '.'], tags=['3']),
 TaggedDocument(words=['Vendedor', 'confiável', ',', 'produto', 'ok', 'e', 'entrega', 'antes', 'do', 'prazo', '.'], tags=['4']),
 TaggedDocument(words=['GOSTARIA', 'DE', 'SABER', 'O', 'QUE', 'HOUVE', ',', 'SEMPRE', 'RECEBI', 'E', 'ESSA', 'COMPRA', 'AGORA', 'ME', 'DECPCIONOU'], tags=['5']),
 T

In [24]:
from gensim.models import Doc2Vec

modelo = Doc2Vec(documents=dados_tag, vector_size=100, min_count=2, window=2, workers=1, seed=45, epochs=20)

In [26]:
vetor_inferido = modelo.infer_vector(['entrega'])

In [27]:
vetor_inferido

array([-0.04966414, -0.06870766,  0.05628688, -0.07012478,  0.03663999,
        0.03859475, -0.01242808, -0.03961292,  0.01815082, -0.08329611,
       -0.08208492,  0.0285581 , -0.07653964,  0.05140475, -0.03174011,
       -0.06455355, -0.01958246,  0.0174532 , -0.00316057,  0.01551982,
        0.0182722 , -0.04131385,  0.06476539, -0.0164116 ,  0.00934094,
       -0.0169923 , -0.08288857, -0.05716019, -0.03443755, -0.02144932,
        0.03199554, -0.00496375,  0.06231426,  0.09313422,  0.0102881 ,
       -0.03857918, -0.07656385,  0.09496396, -0.07553341,  0.02178379,
        0.00039512,  0.05874554,  0.04239239, -0.01302919, -0.000578  ,
       -0.0676636 ,  0.04601046, -0.01020778, -0.04758443, -0.02797714,
        0.00960462, -0.02388016,  0.01057057, -0.02525371,  0.05639075,
       -0.08590603,  0.01457735, -0.01870535, -0.02409266, -0.04604307,
        0.01683144, -0.01282384, -0.01070968, -0.05137849,  0.00167046,
       -0.00955135,  0.05288997,  0.00805376,  0.02147503,  0.01

In [28]:
frases_similares = modelo.dv.most_similar([vetor_inferido], topn=5)
frases_similares

[('9304', 0.8598946928977966),
 ('24666', 0.8501366972923279),
 ('30445', 0.8434346318244934),
 ('24884', 0.8429471254348755),
 ('19676', 0.8409197330474854)]

In [29]:
for idx, similaridade in frases_similares:
    print(f"Review: {dados.iloc[int(idx)]['review']} - Similaridade: {similaridade:.4f}")

Review: entrega no prazo. - Similaridade: 0.8599
Review: entrega antes do prazo - Similaridade: 0.8501
Review: Comprem nas Zoop entrega certaaaa - Similaridade: 0.8434
Review: entrega antes do prazo - Similaridade: 0.8429
Review: Muito boa entrega  - Similaridade: 0.8409


# Aula 2 - Ajustando os textos de review

In [ ]:
dados['review'][11]

In [ ]:
dados['tratamento_1'][11]

## Uniformizando o texto

In [ ]:
!pip install unidecode

## Correções nos dados

In [ ]:
df['review'][120]

In [ ]:
df['tratamento_2'][120]

In [ ]:
df['tratamento_2'][3319]

In [ ]:
df['tratamento_2'][8354]

In [ ]:
df['tratamento_2'][12843]

In [ ]:
df['tratamento_3'][3319]

In [ ]:
df['tratamento_3'][8354]

## Lematização

In [ ]:
!pip install stanza
!python -m stanza.download pt

In [ ]:
def lematizar_texto(textos):
  texto_lematizados = []

  for texto in textos:
    doc_frase = nlp(texto)
    frase_lematizada = ' '.join([palavra.lemma for frase in doc_frase.sentences for palavra in frase.words])
    texto_lematizados.append(frase_lematizada)

  return texto_lematizados

In [ ]:
textos = ['gostei muito experiencia comprar',
          'minha filha gostou produto',
          'compra foi facil compra rapida']


# Aula 3 - Otimizando o Doc2vec

## Treinamento do Doc2vec

## Verificando a similardade - Doc2Vec

# Aula 4 - Considerando o significado das frases

In [ ]:
modulo_url = 'https://tfhub.dev/google/universal-sentence-encoder/4'

## Avaliando as sentenças similares

# Aula 5 - Transformando a similaridade em aplicação

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')

modulo_url = 'https://tfhub.dev/google/universal-sentence-encoder/4'


